In [7]:
import pandas as pd
import os

# Load the CSV file
file_path = "2020-01-27.csv"  
df = pd.read_csv(file_path)

# Convert timestamp column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create an output directory for partitioned files
output_folder = "Hourly_Partitions"
os.makedirs(output_folder, exist_ok=True)

# Group by hour and save each partition separately
for hour in range(24):  
    start_time = df['timestamp'].dt.floor('D') + pd.Timedelta(hours=hour)
    end_time = start_time + pd.Timedelta(hours=1)
    
    # Filter data for the current hour
    hourly_data = df[(df['timestamp'] >= start_time) & (df['timestamp'] < end_time)]
    
    if not hourly_data.empty:
        output_file = os.path.join(output_folder, f"2020-01-27_{hour:02d}.csv")
        hourly_data.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")


Saved: Hourly_Partitions\2020-01-27_00.csv
Saved: Hourly_Partitions\2020-01-27_01.csv
Saved: Hourly_Partitions\2020-01-27_02.csv
Saved: Hourly_Partitions\2020-01-27_03.csv
Saved: Hourly_Partitions\2020-01-27_04.csv
Saved: Hourly_Partitions\2020-01-27_05.csv
Saved: Hourly_Partitions\2020-01-27_06.csv
Saved: Hourly_Partitions\2020-01-27_07.csv
Saved: Hourly_Partitions\2020-01-27_08.csv
Saved: Hourly_Partitions\2020-01-27_09.csv
Saved: Hourly_Partitions\2020-01-27_10.csv
Saved: Hourly_Partitions\2020-01-27_11.csv
Saved: Hourly_Partitions\2020-01-27_12.csv
Saved: Hourly_Partitions\2020-01-27_13.csv
Saved: Hourly_Partitions\2020-01-27_14.csv
Saved: Hourly_Partitions\2020-01-27_15.csv
Saved: Hourly_Partitions\2020-01-27_16.csv
Saved: Hourly_Partitions\2020-01-27_17.csv
Saved: Hourly_Partitions\2020-01-27_18.csv
Saved: Hourly_Partitions\2020-01-27_19.csv
Saved: Hourly_Partitions\2020-01-27_20.csv
Saved: Hourly_Partitions\2020-01-27_21.csv
Saved: Hourly_Partitions\2020-01-27_22.csv
Saved: Hour

In [8]:
import pandas as pd
import os
import json

# Input and Output Folder Paths
input_folder = "Hourly_Partitions"
output_folder = "Hypergraph_JSONs"
os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

# Process each CSV file in the hourly partitions folder
for file_name in sorted(os.listdir(input_folder)):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_folder, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)

        # Create hypergraph dictionary
        hypergraph_data = {}

        for _, row in df.iterrows():
            token = str(row['tokenAddressLabel'])
            from_node = int(row['fromLabel'])
            to_node = int(row['toLabel'])

            if token in hypergraph_data:
                hypergraph_data[token].update([from_node, to_node])  # Add nodes to existing hyperedge
            else:
                hypergraph_data[token] = {from_node, to_node}  # Create a new hyperedge

        # Convert sets to lists for JSON compatibility
        hypergraph_data = {k: list(v) for k, v in hypergraph_data.items()}

        # Output file name
        json_file_name = file_name.replace(".csv", ".json")
        output_path = os.path.join(output_folder, json_file_name)

        # Save hypergraph to JSON file
        with open(output_path, "w") as json_file:
            json.dump(hypergraph_data, json_file, indent=4)

        print(f" Saved: {output_path}")


 Saved: Hypergraph_JSONs\2020-01-27_00.json
 Saved: Hypergraph_JSONs\2020-01-27_01.json
 Saved: Hypergraph_JSONs\2020-01-27_02.json
 Saved: Hypergraph_JSONs\2020-01-27_03.json
 Saved: Hypergraph_JSONs\2020-01-27_04.json
 Saved: Hypergraph_JSONs\2020-01-27_05.json
 Saved: Hypergraph_JSONs\2020-01-27_06.json
 Saved: Hypergraph_JSONs\2020-01-27_07.json
 Saved: Hypergraph_JSONs\2020-01-27_08.json
 Saved: Hypergraph_JSONs\2020-01-27_09.json
 Saved: Hypergraph_JSONs\2020-01-27_10.json
 Saved: Hypergraph_JSONs\2020-01-27_11.json
 Saved: Hypergraph_JSONs\2020-01-27_12.json
 Saved: Hypergraph_JSONs\2020-01-27_13.json
 Saved: Hypergraph_JSONs\2020-01-27_14.json
 Saved: Hypergraph_JSONs\2020-01-27_15.json
 Saved: Hypergraph_JSONs\2020-01-27_16.json
 Saved: Hypergraph_JSONs\2020-01-27_17.json
 Saved: Hypergraph_JSONs\2020-01-27_18.json
 Saved: Hypergraph_JSONs\2020-01-27_19.json
 Saved: Hypergraph_JSONs\2020-01-27_20.json
 Saved: Hypergraph_JSONs\2020-01-27_21.json
 Saved: Hypergraph_JSONs\2020-01

In [9]:
import os
import json
import hypernetx as hnx
import matplotlib.pyplot as plt

# Input and Output Folder Paths
input_folder = "Hypergraph_JSONs"  # Folder containing JSON hypergraphs
output_folder = "Hypergraph_Images"  # Folder to save images
os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

# Process each stored hypergraph JSON file
for file_name in sorted(os.listdir(input_folder)):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)

        # Load JSON hypergraph data
        with open(file_path, "r") as json_file:
            hypergraph_data = json.load(json_file)

        # Create the hypergraph
        H = hnx.Hypergraph(hypergraph_data)

        # Plot the hypergraph
        fig, ax = plt.subplots(figsize=(8, 6))
        hnx.draw(H, with_edge_labels=False, with_node_labels=False, ax=ax)
        plt.title(f"Hypergraph: {file_name.replace('.json', '')}")

        # Save the image
        output_path = os.path.join(output_folder, file_name.replace(".json", ".png"))
        plt.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close()

        print(f" Saved Hypergraph Image: {output_path}")


c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
c:\Users\anand\anaconda3\lib\site-packages\hypernetx

 Saved Hypergraph Image: Hypergraph_Images\2020-01-27_00.png


c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
c:\Users\anand\anaconda3\lib\site-packages\hypernetx

 Saved Hypergraph Image: Hypergraph_Images\2020-01-27_01.png


c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
c:\Users\anand\anaconda3\lib\site-packages\hypernetx

...

 Saved Hypergraph Image: Hypergraph_Images\2020-01-27_21.png


c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
c:\Users\anand\anaconda3\lib\site-packages\hypernetx

 Saved Hypergraph Image: Hypergraph_Images\2020-01-27_22.png


c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:150: DeprecationWarning: misc_props_col and misc_cell_props_col will be deprecated; all public references to these arguments will be removed in a future release.
  warnings.warn(
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])
c:\Users\anand\anaconda3\lib\site-packages\hypernetx\classes\entityset.py:618: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elements = self._dataframe.groupby(col1)[col2].unique().to_dict()
c:\Users\anand\anaconda3\lib\site-packages\hypernetx

 Saved Hypergraph Image: Hypergraph_Images\2020-01-27_23.png


In [11]:
import os
import json
import pandas as pd
from collections import defaultdict

# Input folder containing JSON hypergraphs
input_folder = "Hypergraph_JSONs"

# Output folder for storing influential node results
output_folder = "Influential_Nodes(Hourly)"
os.makedirs(output_folder, exist_ok=True)

# Process each JSON file
for file_name in sorted(os.listdir(input_folder)):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)

        # Load JSON hypergraph data
        with open(file_path, "r") as json_file:
            hypergraph_data = json.load(json_file)

        # Dictionary to store node degrees for this time frame
        node_degree_count = defaultdict(int)

        # Count how many unique hyperedges (tokens) each node is part of
        for token, nodes in hypergraph_data.items():
            for node in nodes:
                node_degree_count[node] += 1  # Increment degree count for this node

        # Convert to DataFrame
        df = pd.DataFrame(list(node_degree_count.items()), columns=["Node", "Degree"])

        # Sort by Degree (Descending), then by Node ID (Ascending)
        df = df.sort_values(by=["Degree", "Node"], ascending=[False, True])

        # Save results to CSV (same name as JSON, but in new folder)
        output_file = os.path.join(output_folder, file_name.replace(".json", ".csv"))
        df.to_csv(output_file, index=False)

print(f"  influential nodes saved in: {output_folder}")


  influential nodes saved in: Influential_Nodes(Hourly)


In [ ]:
import os
import json
import numpy as np
from numpy.linalg import eigh

# Directories
input_folder = "Hypergraph_JSONs"
incidence_folder = "Incidence_Matrices"
degree_folder = "Degree_Matrices"
partition_folder = "Partitioned_Hypergraphs"

os.makedirs(incidence_folder, exist_ok=True)
os.makedirs(degree_folder, exist_ok=True)
os.makedirs(partition_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        
        with open(file_path, "r") as f:
            hyperedges = json.load(f)

        all_nodes = set()
        edge_labels = list(hyperedges.keys())
        for nodes in hyperedges.values():
            all_nodes.update(nodes)
        
        all_nodes = sorted(list(all_nodes))
        node_index = {node: i for i, node in enumerate(all_nodes)}
        
        H = np.zeros((len(all_nodes), len(edge_labels)))
        for j, edge in enumerate(edge_labels):
            for node in hyperedges[edge]:
                H[node_index[node], j] = 1

        d_v = np.sum(H, axis=1)
        D_v = np.diag(d_v)

        d_e = np.sum(H, axis=0)
        D_e = np.diag(d_e)

        with open(os.path.join(incidence_folder, file_name), "w") as f:
            json.dump(H.tolist(), f, indent=4)

        degree_data = {"D_v": D_v.tolist(), "D_e": D_e.tolist()}
        with open(os.path.join(degree_folder, file_name), "w") as f:
            json.dump(degree_data, f, indent=4)

        D_v_inv_sqrt = np.diag(1.0 / np.sqrt(d_v + 1e-10))
        D_e_inv = np.diag(1.0 / (d_e + 1e-10))
        H_term = D_v_inv_sqrt @ H @ D_e_inv @ H.T @ D_v_inv_sqrt
        L = np.eye(len(all_nodes)) - H_term

        eigenvalues, eigenvectors = eigh(L)
        fiedler_vector = eigenvectors[:, 1]

        cluster1 = [node for i, node in enumerate(all_nodes) if fiedler_vector[i] >= 0]
        cluster2 = [node for i, node in enumerate(all_nodes) if fiedler_vector[i] < 0]

        partition_data = {"Cluster_1": cluster1, "Cluster_2": cluster2}
        with open(os.path.join(partition_folder, file_name), "w") as f:
            json.dump(partition_data, f, indent=4)

print(" Process completed! Matrices and partitions stored successfully.")


 Process completed! Matrices and partitions stored successfully.
